# This version

Trying to implement a version about CTC with 3 hidden layer and combine with RNN

***Summary***
1. Try Deep-LSTM by lasagne
- Try dropout layer
- [CTC Problem] prefix search decoding? -> blank probability threshold set at 99.99% at paper
- Try phoneme base instead of char base

***Todo***
1. ~~data normalize~~
2. CTC correct?
- Try phoneme but not character based
1. ~~Using oop module~~
- ~~Using CTC with log domain and recurrent relationship~~
- ~~Add hidden layer before RNN~~
- ~~Compile and update and learning~~
- ~~func check_label_error~~
- ~~data processing to char based~~
    - ~~preprocess of data~~
    - ~~remove_blank(predict)~~
    - ~~remap_back( predict )~~
    - ~~change to tri MFCC only~~
- ~~Try CTC with character based data (data preprocess) (our data)~~
- ~~Check CTC cost correctness~~
- ~~Check CTC with y need to unmap or not~~
- ~~Add Nestrove momentum~~
- ~~Add minibatch~~
- ~~Add bidirectional LSTM/RNN~~

***Further***
1. ~~Add Bidirectional LSTM/RNN~~
2. ~~Make mini batch~~
- Try phoneme based model

***Problem need to understand about RNN***
1. [Phd thesis Training RNN , Hessian free method](http://www.cs.utoronto.ca/~ilya/pubs/ilya_sutskever_phd_thesis.pdf)
- [Does Theano do automatic unfolding for BPTT?](http://stackoverflow.com/questions/24431621/does-theano-do-automatic-unfolding-for-bptt)
- [Github, General purpose Hessian-free optimization in Theano](https://github.com/boulanni/theano-hf)
- [Theano scan](http://deeplearning.net/software/theano/library/scan.html#theano.scan)
- []()

***Problem need to understand about CTC***
1. [CTC paper](ftp://ftp.idsia.ch/pub/juergen/icml2006.pdf)

***Important References***
1. [CTC cost use in our version](https://github.com/mohammadpz/CTC-Connectionist-Temporal-Classification/blob/5b4d4be19805d7795a4293b9c270ef7bf5fafbfc/ctc_cost.py)
    - [Discussion](https://github.com/mohammadpz/CTC-Connectionist-Temporal-Classification/issues/1)
- How to connect multiple recurrent layers,https://github.com/craffel/nntools/issues/9
- https://github.com/craffel/nntools/issues/11
- refactor recurrent, update examples, add tests, https://github.com/craffel/nntools/pull/27
- example of deep lstm and penn tree,https://github.com/skaae/nntools/blob/combine/examples/lstm.py

***Github***
1. [Hessian theano](https://github.com/boulanni/theano-hf/blob/master/hf_examples.py)
- [Vanilla RNN](https://github.com/mohammadpz/Recurrent-Neural-Networks)
- [blocks example, reverse words](https://github.com/mila-udem/blocks-examples/tree/master/reverse_words)
- [rnn-ctc, basics ctc](https://github.com/rakeshvar/rnn_ctc/blob/master/ctc.py)
- [Lasagne nntools](https://github.com/craffel/nntools/tree/recurrent)
- [Theanets, RNN tools](https://github.com/lmjohns3/theanets/blob/master/theanets/layers/recurrent.py)
- [LSTM benchmarks example](https://github.com/craffel/lstm_benchmarks/blob/master/lasagne/experiment.py)

In [1]:
import sys
sys.path.append("../../../../nntools")

In [2]:
import theano
import theano.tensor as T
import numpy as np
from theano_toolkit import utils as U
from theano_toolkit import updates
from theano.printing import Print
from theano_toolkit.parameters import Parameters

import lasagne
from lasagne.layers import *
# from lasagne.layers import RecurrentLayer, InputLayer, DenseLayer,\
#     NonlinearityLayer, ReshapeLayer, EmbeddingLayer
# from lasagne.layers.recurrent import *

from time import time

import ctc_cost_2

import cPickle
import sys
sys.setrecursionlimit(100000)


Using gpu device 0: GeForce GTX 980
/home/pika/nntools/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


# Data part

## read data

In [3]:
### DataProcessing 

'''
[ INPUT  ] : model { wav, NFCC, ... }
[ OUTPUT ] : (Training_data, Validation_data, Testing_Data)

- Training_data : x, y { .astype(np.float32)
- Validation_data : x, y { .astype(np.float32)
- Testing_Data : ( x, id )

-- x format : 
---- MFCC : [ 39-vector ] - 
---- FBank : [ 69-vector ]
---- quadmfcc : [ 39*(1+1+1) - vector ] !!! Wron---- FBank : [ 69-vector ]
---- quadmfcc : [ 39*(1+1+1) - vector ] !!! Wrong implementation
g implementation
---- hexamfcc : [ 39*(4+1+4) - vector ]
---- hexaFbank : [ 69*(4+1+4) - vector ]

-- y format :

---- Now we use 39-phonemes for all : [ 0 0 0 ... 1 .... 0 0 0 ]  as a number of 39
---- But answer we get from ./label is [ 48-vector]
'''


'''
input:
    MFCC: 39,2-1-2
output:
    state label: 1943
'''

'''
[ Input  ] :
    - mfcc (39*(2+1+2))

[ Output ] :
    - maxi (1943-D)
    
'''

from itertools import repeat
import numpy as np
import random
# from sklearn import preprocessing

# Parameter (note: switch /final/MLDS_Final_Data/ .. if u want
MAPPING_FILE = '/home/pika/MLDS_2015/final/MLDS_HW1_RELEASE_v1/phones/48_39.map'
MFCC_TRAINING_ARK = '/home/pika/MLDS_2015/final/MLDS_Final_Data/mfcc/train_nor.ark'
# MFCC_TRAINING_ARK = '/home/pika/MLDS_2015/final/MLDS_Final_Data/wav_mod/train.ark'

# MAXI_TRAINING_LABEL = '/home/pika/MLDS_2015/final/MLDS_HW1_RELEASE_v1/state_label/train.lab'
NORMAL_TRAINING_LABEL = '/home/pika/MLDS_2015/final/MLDS_HW1_RELEASE_v1/label/train.lab'

sentence_data = '/home/pika/MLDS_2015/final/MLDS_Final_Data/sentence/train_mod.set'


def clean_up( y ):
    """
    for final output clean up
    B(a − ab−) = B(−aa − −abb) = aab
    """
    answer = []

    for i in range(len(y)):
        if (i==0):
            answer.append(y[i])
        if y[i-1] != y[i]:
            answer.append(y[i])
    
    return answer



# Dataset 
def DNNDataset( inputMode ) :
    trains = {}
    valids = {}
    test_data = {}
    x_train = []
    x_valid = []
    y_train = []
    y_valid = []
    total_result = []
    
    slide_ration = 0.1
    
    sentense = {}
    char_map = {}
    char_unmap = {}
    char_count = 0
    
    with open(sentence_data) as f:
        for lines in f:
            frames = lines.split(',',1)
            frames[1] = frames[1].lower()
            if len(frames[1])<2: # empty sentences
                prob = frames[0]
                print prob
                continue
            for cha in frames[1][:-2]: # skip "\n" ending punctuation
                if cha in ',.!?;:-"':
                    continue
                if cha not in char_map:
                    char_map[cha] = char_count
                    char_unmap[char_count] = cha
                    char_count += 1
                    
    char_map["-"] = char_count # for blank, different with space
    char_unmap[char_count] = "-"
#     print char_map
    
    with open(sentence_data) as f:
        for lines in f:
            frames = lines.split(',',1)
            frames[1] = frames[1].lower()
            if len(frames[1])<2: # empty sentences
                prob = frames[0]
                continue
            sentense[ frames[0] ] = []
            for cha in frames[1][:-2]:
                if cha in ',.!?;:"':
                    continue
                if cha in '-':
                    cha = " "
#                 sentense[ frames[0] ].append( char_map['-'] ) # append blank before each label
                sentense[ frames[0] ].append( char_map[cha] )
#             sentense[ frames[0] ].append( char_map['-'] ) # append blank in the end

    
    
    
    # X(INPUT) ---------------------------------------
    if inputMode == "mfcc":
        
        # speaker_window -(map)-> mfcc[39]
        monoTrains = {}
        with open(MFCC_TRAINING_ARK) as f:
            for lines in f:
                frames = lines.split(' ')
                frame2float = [ float(x) for x in frames[1:] ]
                monoTrains[frames[0]] = frame2float
#                 break
        # Initialize : L2 L1 S R1 R2
        zeroFrame = list(repeat(0, 39))
        prevframeL1 = [ float(x) for x in zeroFrame[:] ]
#         prevframeL2 = [ float(x) for x in zeroFrame[:] ]
        prevframeR1 = [ float(x) for x in zeroFrame[:] ]
#         prevframeR2 = [ float(x) for x in zeroFrame[:] ]
        blankframe  = [ float(x) for x in zeroFrame[:] ]
        frameNameL1 = ""
#         frameNameL2 = ""
        frameNameR1 = ""
#         frameNameR2 = ""
        
        # Speaker { windows { ... } }
        # still is faem0_si1392, bug !
        
        speaker_data = []
        for sample in monoTrains:
            frames_info = sample.split('_')
            if frames_info[0] not in speaker_data:
                speaker_data.append(frames_info[0])

        speaker_data = np.array(speaker_data)
        speaker_data = np.random.permutation(speaker_data)
#         print speaker_data
        valid = len(speaker_data)*0.9
        speaker_train = speaker_data[ 0 : valid ]
        speaker_valid = speaker_data[ valid : ]
        
        for sample in monoTrains:
            frames_info = sample.split('_')
            speaker = frames_info[0] + "_" + frames_info[1]
#             frameNameL2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-2))
            frameNameL1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-1))
            frameNameR1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+1))
#             frameNameR2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+2))
            
            if frames_info[0] in speaker_train:
                
                if trains.get(speaker) is None:
                    trains[speaker] = {}
                if   frames_info[2] == "1":
                    trains[speaker][sample] = blankframe + monoTrains[sample] + monoTrains[frameNameR1]
                elif monoTrains.get(frameNameR1) is None:
                    trains[speaker][sample] = monoTrains[frameNameL1] + monoTrains[sample] + blankframe
                else:
                    trains[speaker][sample] = monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1]  
#                 trains[speaker][sample] = monoTrains[sample]
            else:
                if valids.get(speaker) is None:
                    valids[speaker] = {}
                if   frames_info[2] == "1":
                    valids[speaker][sample] = blankframe + monoTrains[sample] + monoTrains[frameNameR1]

                elif monoTrains.get(frameNameR1) is None:
                    valids[speaker][sample] = monoTrains[frameNameL1] + monoTrains[sample] + blankframe
                else:
                    valids[speaker][sample] = monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1]
#                 valids[speaker][sample] = monoTrains[sample]

        monoTrains.clear()
        
        phones_mapping = {}
        result_mapping = {}
        y_trains_phone = {}
        with open(MAPPING_FILE) as f:
            i = 0
            for lines in f :
                phones = lines.split('\t')
                phones_mapping[phones[0]] = i
                i += 1
                
        with open(MAPPING_FILE) as f:    
            for lines in f :
                phones = lines.split('\t')
                result_mapping[ phones_mapping[phones[0]] ] = phones[1]
        
        record_speaker = []
        with open(NORMAL_TRAINING_LABEL) as f:
            for lines in f:
                # print lines
                byenextline = lines.split('\n')
                label_info = byenextline[0].split(',')
                frame_info = label_info[0].split('_')
                speaker = frame_info[0] + "_" + frame_info[1]
                if (speaker not in y_trains_phone):
                    y_trains_phone[ speaker ] = []
                    record_speaker.append(speaker)
                y_trains_phone[ speaker ].append( phones_mapping[label_info[1].rstrip('\n')] ) # a class number is enough
                
#         print record_speaker
        for sss in record_speaker:
#             print y_trains_phone[sss]
            y_trains_phone[sss] = clean_up(y_trains_phone[sss])
#             print y_trains_phone[sss]
#             break
                        
        
        for sample in trains:
#             print sample
#             print len(trains[sample])
            
            if sample in sentense:
                temp = []
                for i in range(len(trains[sample])):
                    temp.append( floatX( trains[sample][sample + "_" + str(i+1)] ))
                    

                x_train.append( floatX(temp ))
                
                y_train.append(floatX(y_trains_phone[sample]))

#                 y_train.append( floatX(sentense[sample]) )

        trains.clear()
        
        for sample in valids:
#             print sample
#             print len(trains[sample])
            if sample in sentense:
                temp = []
                for i in range(len(valids[sample])):
                    ( temp.append( floatX ( valids[sample][sample + "_" + str(i+1)]) ))
                x_valid.append( floatX(temp ))
                y_valid.append(floatX(y_trains_phone[sample]))
                
    
# #     print "normalized"
# #     x_train = preprocessing.scale( x_train)
# #     x_valid = preprocessing.scale( x_valid)
    
#     trains.clear()
#     valids.clear()
        
    return char_map, char_unmap, phones_mapping, result_mapping, x_train, y_train, x_valid, y_valid

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def vectorized_result ( j , siz) :
    e = np.zeros((siz, 1))
    e[j] = 1.0
    return np.reshape( e, siz)


In [4]:
char_map, char_unmap, phones_mapping, result_mapping, x_train, y_train, x_valid, y_valid = DNNDataset("mfcc")

mcth0_sx219


In [ ]:
print y_train[0]
# x_train[0][0].shape[0]
print (char_unmap)
print len(char_unmap)

print len(x_train)
print len(x_train[0])
print x_train[0][0].shape

In [ ]:
# print x_train.shape
# print x_train[0].shape
# print x_train[0][1]
len(x_train[1])

In [ ]:
# print x_test[0].shape
# print x_test[0][1][0]
len(phones_mapping)

# Data part

## make batch

In [5]:
def make_batches_X(X, length, batch_size=30):
    '''
    https://github.com/craffel/lstm_benchmarks/blob/master/lasagne/experiment.py
    
    Convert a list of matrices into batches of uniform length
    :parameters:
        - X : list of np.ndarray
            List of matrices
        - length : int
            Desired sequence length.  Smaller sequences will be padded with 0s,
            longer will be truncated.
        - batch_size : int
            Mini-batch size
    :returns:
        - X_batch : np.ndarray
            Tensor of time series matrix batches,
            shape=(n_batches, length, batch_size, n_features)
        - X_mask : np.ndarray
            shape=(n_batches, length, batch_size)
            Mask denoting whether to include each time step of each time series
            matrix
    '''
    n_batches = len(X)//batch_size
    
    # Lasagne format
    X_batch = np.zeros((n_batches, batch_size, length, X[0].shape[1]),
                       dtype=theano.config.floatX)
    # Else format
#     X_batch = np.zeros((n_batches, length, batch_size, X[0].shape[1]),
#                        dtype=theano.config.floatX)
    
#     X_mask = np.zeros(X_batch.shape, dtype=np.bool)
    
    X_mask = np.zeros((n_batches, length, batch_size ), dtype=theano.config.floatX)
#     X_mask = np.zeros((n_batches,  batch_size, length ), dtype=theano.config.floatX)

    
    
    for b in range(n_batches): 
        for n in range(batch_size): # go thorough batch size
            
            X_m = X[b*batch_size + n] # seq_length X feature dim
            
            X_batch[b, n, :X_m.shape[0]] = X_m[:length]
#             X_batch[b, :X_m.shape[0], n] = X_m[:length]
            
#             X_mask[b, n, :X_m.shape[0]] = 1
            X_mask[b, :X_m.shape[0], n] = 1
            
    return X_batch, X_mask

def make_batches_Y( X, length, batch_size=30):
    '''
    https://github.com/craffel/lstm_benchmarks/blob/master/lasagne/experiment.py
    
    Convert a list of matrices into batches of uniform length
    :parameters:
        - X : list of np.ndarray
            List of matrices
        - length : int
            Desired sequence length.  Smaller sequences will be padded with 0s,
            longer will be truncated.
        - batch_size : int
            Mini-batch size
    :returns:
        - X_batch : np.ndarray
            Tensor of time series matrix batches,
            shape=(n_batches, length, batch_size )
        - X_mask : np.ndarray
            Mask denoting whether to include each time step of each time series
            matrix
    '''
    n_batches = len(X)//batch_size
    
    X_batch = np.zeros( (n_batches, length, batch_size ), dtype='float32')
    
    X_mask = np.zeros(X_batch.shape, dtype=theano.config.floatX)
    
    for b in range(n_batches):
        for n in range(batch_size):
            X_m = X[ b*batch_size + n ]
            X_batch[b, :X_m.shape[0], n ] = X_m[:length]
            X_mask[b, :X_m.shape[0], n] = 1
    return X_batch, X_mask


In [6]:
batch_size = 32

# Find the longest sequence
length_x = max(max([X.shape[0] for X in x_train]),
             max([X.shape[0] for X in x_valid]))
length_y = max(max([X.shape[0] for X in y_train]),
             max([X.shape[0] for X in y_valid]))

# Convert to batches of time series of uniform length
# x_train_mask: seq_length X batch_size
x_train, x_train_mask = make_batches_X(x_train, length_x, batch_size)
# y_train_mask: output_length X batch_size
# y_pred_mask = x_train_mask, since pred by sequence
y_train, y_train_mask = make_batches_Y(y_train, length_y, batch_size)
x_valid, x_valid_mask = make_batches_X(x_valid, length_x, batch_size)
y_valid, y_valid_mask = make_batches_Y(y_valid, length_y, batch_size)

# x_train_t, x_train_mask = make_batches_X(x_train, length_x, batch_size)
# # y_train_mask: output_length X batch_size
# # y_pred_mask = x_train_mask, since pred by sequence
# y_train_t, y_train_mask = make_batches_Y(y_train, length_y, batch_size)
# x_valid_t, x_valid_mask = make_batches_X(x_valid, length_x, batch_size)
# y_valid_t, y_valid_mask = make_batches_Y(y_valid, length_y, batch_size)


In [7]:
print x_train.shape
print x_valid.shape
print y_train.shape
print y_valid.shape
print "================mask================"
print x_train_mask.shape
print y_valid_mask.shape

# print x_train_mask[0][400][0]

(103, 32, 777, 117)
(11, 32, 777, 117)
(103, 74, 32)
(11, 74, 32)
================mask================
(103, 777, 32)
(11, 74, 32)


In [ ]:
y_train[0][20]

In [ ]:
a = np.array([1,2,3,4,5])
b = np.array([1,0,0,0,1])
a[np.nonzero(b)]


# Data part

## decode for phone based

In [3]:
'''
credit by NLTK package of measure edit distance
'''

def _edit_dist_init(len1, len2):
    lev = []
    for i in range(len1):
        lev.append([0] * len2)  # initialize 2D array to zero
    for i in range(len1):
        lev[i][0] = i           # column 0: 0,1,2,3,4,...
    for j in range(len2):
        lev[0][j] = j           # row 0: 0,1,2,3,4,...
    return lev


def _edit_dist_step(lev, i, j, s1, s2, transpositions=False):
    c1 = s1[i - 1]
    c2 = s2[j - 1]

    # skipping a character in s1
    a = lev[i - 1][j] + 1
    # skipping a character in s2
    b = lev[i][j - 1] + 1
    # substitution
    c = lev[i - 1][j - 1] + (c1 != c2)

    # transposition
    d = c + 1  # never picked by default
    if transpositions and i > 1 and j > 1:
        if s1[i - 2] == c2 and s2[j - 2] == c1:
            d = lev[i - 2][j - 2] + 1

    # pick the cheapest
    lev[i][j] = min(a, b, c, d)


def check_label_error( real , predict, transpositions=False):
    ## length of real >= length of predict
    """
    Calculate the Levenshtein edit-distance between two strings.
    The edit distance is the number of characters that need to be
    substituted, inserted, or deleted, to transform s1 into s2.  For
    example, transforming "rain" to "shine" requires three steps,
    consisting of two substitutions and one insertion:
    "rain" -> "sain" -> "shin" -> "shine".  These operations could have
    been done in other orders, but at least three steps are needed.

    This also optionally allows transposition edits (e.g., "ab" -> "ba"),
    though this is disabled by default.

    :param s1, s2: The strings to be analysed
    :param transpositions: Whether to allow transposition edits
    :type s1: str
    :type s2: str
    :type transpositions: bool
    :rtype int
    """
    # set up a 2-D array
    len1 = len(predict)
    len2 = len(real)
    lev = _edit_dist_init(len1 + 1, len2 + 1)

    # iterate over the array
    for i in range(len1):
        for j in range(len2):
            _edit_dist_step(lev, i + 1, j + 1, predict, real, transpositions=transpositions)
            
    return lev[len1][len2]*1.0/len2


def clean_up_phone( y ):
    """
    for final output clean up
    B(a − ab−) = B(−aa − −abb) = aab
    """
    answer = []

    for i in range(len(y)):
        
        if y[i] == len(phones_mapping): # blank
                continue
        if i != 0:
            if y[i-1] != y[i]:
                answer.append(y[i])
        else:
            answer.append(y[i])
    
    return answer


In [4]:

from sklearn.metrics import accuracy_score

def check_phone_err(y_actual, y_pred, y_mask_actual, y_mask_pred, batch_size):
    
    err = 0.
    for i in xrange(batch_size):
#         print i
        mask_actual = np.swapaxes( y_mask_actual, 0, 1)[i]
        ans_actual = np.swapaxes( y_actual, 0 , 1)[i]
        mask_pred = np.swapaxes( y_mask_pred, 0, 1)[i]
        ans_pred = y_pred[i]
        
        y_rrr = ans_actual[ np.nonzero(mask_actual) ]
        y_ppp = ans_pred[ np.nonzero(mask_pred) ]
        
        y_ppp = clean_up_phone( y_ppp )
        
#         print "Target ",y_rrr
#         print "\t=>", y_ppp
        
        err += check_label_error(y_rrr, y_ppp)
        
#         score += accuracy_score(y_true, y_pred)
    
    return err
    

In [ ]:
a = np.array([0,0,1,2,3,3,3,48,2,2,3,3,2,3,3,4])
b = np.array([48,48,1,1,2,3,3,48,2,2,3,2,3,3,4,5])

# print clean_up_phone(b)
check_label_error(a,b)

In [ ]:
x_train_mask[0].shape

In [ ]:
# check_phone_err(y_actual, y_pred, y_mask_actual, y_mask_pred, batch_size):
print np.argmax(abc[0],axis=2).shape
check_phone_err(y_train[0], np.argmax(abc[0],axis=2), y_train_mask[0], x_train_mask[0],  batch_size)

# Model part

In [5]:

class end_to_end():
    
    def __init__(self, input_shape, max_seq_length, hidden_layer, rnn_hidden_layer,
                 batch, max_epochs, eval_size, output_num_units,
                 patience, up_learning_rate, up_momentum, file_name):
        
        self.input_shape = input_shape # [batch, dim]
        self.max_seq_length = max_seq_length
        self.hidden_layer = hidden_layer # hidden [l1, l2, l3]
        self.rnn_hidden_layer = rnn_hidden_layer # rnn hidden [l1]
        self.output_num_units = output_num_units # [ # of class ]
        
        self.batch = batch
        self.max_epochs = max_epochs
        self.eval_size = eval_size
        
        self.up_learning_rate = up_learning_rate
        self.up_momentum = up_momentum
                
        self.patience = patience
        self.best_valid = np.inf
        self.best_valid_epoch = 0
        self.best_params = None
        
        self.train_history_ = []
        
        self.file_name = file_name
        
        self.drop_frac = 0.2
#         X = T.matrix('X')
#         Y = T.ivector('Y')     
        
        """
        input data type
        y_hat : T x B x C+1
        y : L x B
        y_hat_mask : T x B
        y_mask : L x B
        """
        
        # T x B x F
        # B X T X F (Lasagne format)
        x = T.tensor3('X', dtype=theano.config.floatX)
        # L x B
        y = T.matrix('y', dtype=theano.config.floatX)

        # L x B
        y_mask = T.matrix('y_mask', dtype=theano.config.floatX)
        # T x B
        y_hat_mask = T.matrix('y_hat_mask', dtype=theano.config.floatX)
    
        # Min/max sequence length
#         MIN_LENGTH = 50
        MAX_LENGTH = max_seq_length
        # Number of units in the hidden (recurrent) layer
        N_HIDDEN = hidden_layer[0]
        # Number of training sequences in each batch
        N_BATCH = batch
        # Optimization learning rate
        GRAD_CLIP = 100
        # How often should we check the output?
        EPOCH_SIZE = 100
        # Number of epochs to train the net
        NUM_EPOCHS = 10
        
        #===========================================================================================
        # Bi RNN
        #===========================================================================================

        # Recurrent layers expect input of shape
        # (batch size, max sequence length, number of features)
        # T x B x F
#         l_in = lasagne.layers.InputLayer(shape=( N_BATCH, MAX_LENGTH,  self.input_shape[1]))
        
        # We're using a bidirectional network, which means we will combine two
        # RecurrentLayers, one with the backwards=True keyword argument.
        # Setting a value for grad_clipping will clip the gradients in the layer
#         l_forward = lasagne.layers.RecurrentLayer(l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
#                                                   W_in_to_hid=lasagne.init.HeUniform(),
#                                                   W_hid_to_hid=lasagne.init.HeUniform(),
#                                                   nonlinearity=lasagne.nonlinearities.tanh)
#         l_backward = lasagne.layers.RecurrentLayer(l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
#                                                    W_in_to_hid=lasagne.init.HeUniform(),
#                                                    W_hid_to_hid=lasagne.init.HeUniform(),
#                                                    nonlinearity=lasagne.nonlinearities.tanh, 
#                                                    backwards=True)
        # We'll use an elementwise sum to combine the forward/backward layers
#         l_sum = lasagne.layers.ElemwiseSumLayer([l_forward, l_backward])
        
        # We need a reshape layer which combines the first (batch size) and second
        # (number of timesteps) dimensions, otherwise the DenseLayer will treat the
        # number of time steps as a feature dimension
#         l_reshape = lasagne.layers.ReshapeLayer( l_sum, (N_BATCH*MAX_LENGTH, N_HIDDEN))
        
        #===========================================================================================
        # dEEP lSTM
        #===========================================================================================
        
        # Recurrent layers expect input of shape
        # (batch size, max sequence length, number of features)
        l_in = lasagne.layers.InputLayer(shape=(N_BATCH, MAX_LENGTH,  self.input_shape[1]))
        
        l_in_gau = lasagne.layers.GaussianNoiseLayer(l_in, sigma=0.5, )
        
        # LSTM layer 1
        l_forward_1 = lasagne.layers.LSTMLayer(l_in_gau, num_units=hidden_layer[0], 
                                               learn_init=True, peepholes=True)
        l_backward_1 = lasagne.layers.LSTMLayer(l_in, num_units=hidden_layer[0],
                                                backwards=True, learn_init=True, peepholes=True)
        l_recurrent_1 = ElemwiseSumLayer([l_forward_1, l_backward_1])
  
        # concatenate forward and backward LSTM layers
#         l_fwd_reshape_1 = lasagne.layers.ReshapeLayer(l_forward_1, (N_BATCH*MAX_LENGTH, hidden_layer[0]))
#         l_bck_reshape_1 = lasagne.layers.ReshapeLayer(l_backward_1, (N_BATCH*MAX_LENGTH, hidden_layer[0]))
#         l_recurrent_1 = lasagne.layers.ConcatLayer([l_fwd_reshape_1, l_bck_reshape_1], axis=1)
        
#         if self.drop_frac > 0:
#             l_drp_1 = lasagne.layers.DropoutLayer(l_recurrent_1, p = 0.2 )
#         else:
#             l_drp_1 = l_recurrent_1

        # LSTM layer 2
        l_forward_2 = lasagne.layers.LSTMLayer(l_recurrent_1, num_units=hidden_layer[1], 
                                               learn_init=True, peepholes=True)
        l_backward_2 = lasagne.layers.LSTMLayer(l_recurrent_1, num_units=hidden_layer[1],
                                                backwards=True, learn_init=True, peepholes=True)
        l_recurrent_2 = ElemwiseSumLayer([l_forward_2, l_backward_2])
        
#         if self.drop_frac > 0:
#             l_drp_2 = lasagne.layers.DropoutLayer(l_recurrent_2, p = 0.2 )
#         else:
#             l_drp_2 = l_recurrent_2
        
        # LSTM layer 3
        l_forward_3 = lasagne.layers.LSTMLayer(l_recurrent_2, num_units=hidden_layer[2], 
                                               learn_init=True, peepholes=True)
        l_backward_3 = lasagne.layers.LSTMLayer(l_recurrent_2, num_units=hidden_layer[2],
                                                backwards=True, learn_init=True, peepholes=True)
        l_recurrent_3 = ElemwiseSumLayer( [l_forward_3, l_backward_3] )
        
#         if self.drop_frac > 0:
#             l_drp_3 = lasagne.layers.DropoutLayer(l_recurrent_3, p = 0.2 )
#         else:
#             l_drp_3 = l_recurrent_3

        
        # LSTM layer 4
        l_forward_4 = lasagne.layers.LSTMLayer(l_recurrent_3, num_units=hidden_layer[3], 
                                               learn_init=True, peepholes=True)
        l_backward_4 = lasagne.layers.LSTMLayer(l_recurrent_3, num_units=hidden_layer[3],
                                                backwards=True, learn_init=True, peepholes=True)
        l_recurrent_4 = ElemwiseSumLayer( [l_forward_4, l_backward_4] )

#         if self.drop_frac > 0:
#             l_drp_4 = lasagne.layers.DropoutLayer(l_recurrent_4, p = 0.2 )
#         else:
#             l_drp_4 = l_recurrent_4
            
        # LSTM layer 5
#         l_forward_5 = lasagne.layers.LSTMLayer(l_drp_4, num_units=hidden_layer[4], 
#                                                learn_init=True, peepholes=True)
#         l_backward_5 = lasagne.layers.LSTMLayer(l_drp_4, num_units=hidden_layer[4],
#                                                 backwards=True, learn_init=True, peepholes=True)
#         l_recurrent_5 = ElemwiseSumLayer( [l_forward_5, l_backward_5] )

#         if self.drop_frac > 0:
#             l_drp_5 = lasagne.layers.DropoutLayer(l_recurrent_5, p = 0.2 )
#         else:
#             l_drp_5 = l_recurrent_5


        
        l_reshape = lasagne.layers.ReshapeLayer(l_recurrent_4,
                                               (N_BATCH*MAX_LENGTH, hidden_layer[3]))

        
        #===========================================================================================
        # COMMON SETUP
        #===========================================================================================
        
        # Our output layer is a simple dense connection
        l_recurrent_out = lasagne.layers.DenseLayer( l_reshape, num_units= output_num_units[0] ,
                                                    nonlinearity=lasagne.nonlinearities.identity)
        
        # Now, reshape the output back to the RNN format
        l_out_shp = lasagne.layers.ReshapeLayer(l_recurrent_out, (N_BATCH, MAX_LENGTH, output_num_units[0] ))
        
        # dimshuffle to shape format (input_seq_len, batch_size, num_classes + 1)
        l_out_shp_ctc = lasagne.layers.DimshuffleLayer(l_out_shp, (1, 0, 2))

        l_out_softmax = lasagne.layers.NonlinearityLayer(
                            l_recurrent_out, nonlinearity=lasagne.nonlinearities.softmax)
        l_out_softmax_shp = lasagne.layers.ReshapeLayer(
                            l_out_softmax, (N_BATCH, MAX_LENGTH, output_num_units[0] ))
        
        # since we use gaussian noise in input, False means use noise, True means dont use noise
        output_lin_ctc_train = lasagne.layers.get_output(l_out_shp_ctc, x, deterministic=False)
        output_softmax_train = lasagne.layers.get_output(l_out_softmax_shp, x, deterministic=False)
        
        output_lin_ctc_val = lasagne.layers.get_output(l_out_shp_ctc, x, deterministic=True)
        output_softmax_val = lasagne.layers.get_output(l_out_softmax_shp, x, deterministic=True)
        
        self.all_params = lasagne.layers.get_all_params(l_out_shp)

        # the CTC cross entropy between y and linear output network
        pseudo_cost = ctc_cost_2.pseudo_cost(
            y, output_lin_ctc_train, y_mask, y_hat_mask,
            skip_softmax=True)
        
        
        pseudo_cost_grad = T.grad(pseudo_cost.mean(), self.all_params)
        true_cost = ctc_cost_2.cost(y, output_softmax_train.dimshuffle(1, 0, 2), y_mask, y_hat_mask)
        cost = T.mean(true_cost)
        updates = lasagne.updates.rmsprop(pseudo_cost_grad, self.all_params, learning_rate=0.0001)

        self.train = theano.function(
            inputs = [x, y, y_hat_mask, y_mask],
            outputs = [ pseudo_cost.mean(), cost, output_softmax_train ],
#             outputs = [output_lin_ctc, output_softmax, cost],
            updates=updates
        )
        
        self.predict = theano.function( 
            inputs=[x], 
            outputs = [ output_softmax_val] 
        )

        
    # x_mask and y_mask is the same
    # x_test_mask and y_test_mask is the same
    def fit(self, x_train, y_train, x_test,  y_test , x_mask, y_mask, x_test_mask, y_test_mask ):
        print " "
        print "start training!!!!"
        print " "
        
        epochs = 0
        for i in range(self.max_epochs):
            epochs +=1
            t0 = time()
            
            cs = 0.
            pseudo_cs = 0.
            for index in range(len(x_train)):
                pseudo, cost, output_softmax = self.train( x_train[index] , y_train[index],
                                                          x_mask[index], y_mask[index])
                cs += cost
                pseudo_cs += pseudo
                gg = index
#                 print pseudo, cost
                
            cs /= len(x_train)
            pseudo_cs /= len(x_train)
            
#             cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
            if epochs <= 1:
                previous_cs = "-"
            else:
                previous_cs = self.train_history_[-1]['cost']
            print "\n==============================="
            print 'epoch {0} : pseudo= {1}, cost= {2}, previous_cost= {3}, train_time = {4} s'.format(epochs,
                                                                                                      pseudo_cs, 
                                                                                                      cs,
                                                                                                      previous_cs, 
                                                                                                      time() - t0)
            
            sys.stdout.flush()
            
            t0 = time()
            
#             print np.argmax(output_softmax[:],axis=2)
#             print decode_all( np.argmax(output_softmax[:],axis=2) , x_train_mask[gg],  batch_size , True)
            
            # save model first
            # dont know the LER performance yet
#             cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
    

            err = 0.
            for index in range( len(x_test) ) :

                prepre = self.predict( x_test[index])
                
                err += check_phone_err(y_test[index], np.argmax(prepre[0],axis=2), 
                                           y_test_mask[index], x_test_mask[index], batch_size)
#                 label_error_rate += check_all( y_test[index], y_test_mask[index],
#                                               np.argmax(prepre[0],axis=2) , x_test_mask[index],
#                                               batch_size)
                
#                 if (( index + epochs ) % 4 == 0 ):
#                     ## print actual
#                     y_actual = decode_all_actual( y_test[index], y_test_mask[index], batch_size , True)

#                     ## print pred
#                     y_predict = decode_all_pred( np.argmax(prepre[0],axis=2) , x_test_mask[index],  batch_size , False)
                
            err /= len(x_test)
            self.train_history_.append({"epoch":epochs, "cost": cs, "err":err})
            
            print '\t\t\t val_= {0}, test_time  = {1} s'.format(err, time() - t0)
            print "===============================\n"
            
            sys.stdout.flush()
            
#             for a , b in zip (y_actual,y_predict):
#                 print a 
#                 print "====>>",b
            
#             sys.stdout.flush()
            
            """
            should use cost do early stopping
            """
            current_cs = self.train_history_[-1]['cost']
            current_epoch = self.train_history_[-1]['epoch']
            if current_cs < self.best_valid:
                self.best_valid = current_cs
                self.best_valid_epoch = current_epoch
                self.best_params = [w.get_value() for w in self.all_params]
                cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
                
            elif self.best_valid_epoch + self.patience <= current_epoch:
                print ""
                print "Early stopping."
                print self.best_valid_epoch,self.best_valid
                print "Best valid ler {:.6f} at epoch {}.".format(self.best_valid, self.best_valid_epoch)              
#                 for qq in range (len(self.all_params)):
#                     self.all_params[qq].set_value( self.best_params[qq] )
#                 break

            """
            can not use label error rate do early stopping
            """
#             current_ler = self.train_history_[-1]['LER']
#             current_epoch = self.train_history_[-1]['epoch']
#             if current_ler < self.best_valid:
# #                 print "********************* Now best ************************"
#                 sys.stdout.flush()
#                 self.best_valid = current_ler
#                 self.best_valid_epoch = current_epoch
#                 self.best_params = [w.get_value() for w in self.all_params]
# #                 cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
                
#             elif self.best_valid_epoch + self.patience <= current_epoch:
#                 print ""
#                 print "Early stopping."
#                 print self.best_valid_epoch,self.best_valid
#                 print "Best valid ler {:.6f} at epoch {}.".format(self.best_valid, self.best_valid_epoch)
#                 sys.stdout.flush()                
#                 for qq in range (len(self.all_params)):
#                     self.all_params[qq].set_value( self.best_params[qq] )
#                 break


#     def prediction(self, x, x_mask) :
        
#         abc =  self.predict(x, x_mask)
        
#         return np.argmax(abc[0], axis = 1 )



# Experiment part

In [6]:
num_classes = len(char_unmap) - 2
blank_symbol = len(char_unmap)- 1
print num_classes
print blank_symbol

NameError: name 'char_unmap' is not defined

In [9]:
model2 = end_to_end (
    input_shape      = (1, x_train.shape[3] ) , # batch of 1, (110, 30, 777, 117) 103, 32, 980, 128
#     input_shape = (1, 117),
    max_seq_length = x_train.shape[2],
    hidden_layer     = [ 250, 250 , 250, 250, 200 ], # maximum layer to LSTM 3 layer only
    rnn_hidden_layer = [ 100 ], # maximum layer to 1 layer only
    batch            = batch_size, 
    max_epochs       = 300, 
    eval_size        = 0.1, 
    output_num_units = [ len(phones_mapping) + 1 ],
    up_learning_rate = 0.1, 
    up_momentum      = 0.9, 
    patience         = 5,
    file_name = "model_phone_1"
)


/home/pika/nntools/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "
/home/pika/anaconda/lib/python2.7/site-packages/theano/scan_module/scan.py:1017: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)
/home/pika/anaconda/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


KeyboardInterrupt: 

In [ ]:
for qq in range (len(model1.all_params)):
    model2.all_params[qq].set_value( model1.all_params[qq].get_value() )


In [ ]:
# x_mask and y_mask is the same
# x_test_mask and y_test_mask is the same

%time model2.fit( x_train, y_train, x_valid, y_valid, x_train_mask, y_train_mask , x_valid_mask, y_valid_mask )

# Model 3 testing

## with validation data set, with proper predict

### set model 2 result to model 3 first

In [12]:
model1 = end_to_end (
    input_shape      = (1, x_train.shape[3] ) , # batch of 1, (110, 30, 777, 117) 103, 32, 980, 128
#     input_shape = (1, 117),
    max_seq_length = x_train.shape[2],
    hidden_layer     = [ 250, 250 , 250, 250, 200 ], # maximum layer to LSTM 3 layer only
    rnn_hidden_layer = [ 100 ], # maximum layer to 1 layer only
    batch            = batch_size, 
    max_epochs       = 300, 
    eval_size        = 0.1, 
    output_num_units = [ len(phones_mapping) + 1 ],
    up_learning_rate = 0.1, 
    up_momentum      = 0.9, 
    patience         = 5,
    file_name = "model_phone_1"
)


/home/pika/nntools/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "
/home/pika/anaconda/lib/python2.7/site-packages/theano/scan_module/scan.py:1017: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)
/home/pika/anaconda/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [42]:
abc = model1.predict( x_train[0] )

MemoryError: Error allocating 11636352 bytes of device memory (out of memory).
Apply node that caused the error: GpuFromHost(X)
Inputs types: [TensorType(float32, 3D)]
Inputs shapes: [(32, 777, 117)]
Inputs strides: [(363636, 468, 4)]
Inputs values: ['not shown']

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [ ]:
# check_phone_err(y_actual, y_pred, y_mask_actual, y_mask_pred, batch_size):
print np.argmax(abc[0],axis=2).shape
check_phone_err(y_train[0], np.argmax(abc[0],axis=2), y_train_mask[0], x_train_mask[0],  batch_size)

In [ ]:
%time model1.fit( x_train, y_train, x_valid, y_valid, x_train_mask, y_train_mask , x_valid_mask, y_valid_mask )

In [54]:
cPickle.dump( net5_see_see , open("./model_phone_1_poor.pkl","wb"))

In [52]:
net5_see_see.train_history_

[{'cost': inf, 'epoch': 1, 'err': 31.991835016835022},
 {'cost': inf, 'epoch': 2, 'err': 31.833766586503668},
 {'cost': inf, 'epoch': 3, 'err': 31.990362297277194},
 {'cost': 158.0561217521001, 'epoch': 4, 'err': 31.910177979695053},
 {'cost': 145.73339465983864, 'epoch': 1, 'err': 31.15705348607041},
 {'cost': 149.67699106457164, 'epoch': 2, 'err': 31.068273132204926},
 {'cost': 137.5012452949598, 'epoch': 3, 'err': 31.068467382399174},
 {'cost': 133.60652227309143, 'epoch': 4, 'err': 31.07160217863742},
 {'cost': 129.87853915020099, 'epoch': 5, 'err': 31.065869979801775},
 {'cost': 125.51542426544486, 'epoch': 6, 'err': 31.068467382399174},
 {'cost': 118.49587975659416, 'epoch': 7, 'err': 31.059983174655027},
 {'cost': 109.07098907174417, 'epoch': 8, 'err': 30.968467507847684},
 {'cost': 101.07294819655928, 'epoch': 9, 'err': 30.368506854381739},
 {'cost': 94.865259781624502, 'epoch': 10, 'err': 29.317447069115506},
 {'cost': 86.062599182128906, 'epoch': 11, 'err': 27.279360794857947

In [55]:
net5_see_see.train_history_

[{'cost': inf, 'epoch': 1, 'err': 31.991835016835022},
 {'cost': inf, 'epoch': 2, 'err': 31.833766586503668},
 {'cost': inf, 'epoch': 3, 'err': 31.990362297277194},
 {'cost': 158.0561217521001, 'epoch': 4, 'err': 31.910177979695053},
 {'cost': 145.73339465983864, 'epoch': 1, 'err': 31.15705348607041},
 {'cost': 149.67699106457164, 'epoch': 2, 'err': 31.068273132204926},
 {'cost': 137.5012452949598, 'epoch': 3, 'err': 31.068467382399174},
 {'cost': 133.60652227309143, 'epoch': 4, 'err': 31.07160217863742},
 {'cost': 129.87853915020099, 'epoch': 5, 'err': 31.065869979801775},
 {'cost': 125.51542426544486, 'epoch': 6, 'err': 31.068467382399174},
 {'cost': 118.49587975659416, 'epoch': 7, 'err': 31.059983174655027},
 {'cost': 109.07098907174417, 'epoch': 8, 'err': 30.968467507847684},
 {'cost': 101.07294819655928, 'epoch': 9, 'err': 30.368506854381739},
 {'cost': 94.865259781624502, 'epoch': 10, 'err': 29.317447069115506},
 {'cost': 86.062599182128906, 'epoch': 11, 'err': 27.279360794857947

In [53]:
# x_mask and y_mask is the same
# x_test_mask and y_test_mask is the same

%time net5_see_see.fit( x_train, y_train, x_valid, y_valid, x_train_mask, y_train_mask , x_valid_mask, y_valid_mask )

 
start training!!!!
 

epoch 1 : pseudo= -7.40411556346, cost= 20.9691135906, previous_cost= -, train_time = 1261.18023181 s
			 val_= 10.1964500271, test_time  = 11.3013911247 s


epoch 2 : pseudo= -7.56373995253, cost= 20.1380412, previous_cost= 20.9691135906, train_time = 1257.9420011 s
			 val_= 10.1695462635, test_time  = 11.279168129 s


epoch 3 : pseudo= -7.59256823086, cost= 19.3699461743, previous_cost= 20.1380412, train_time = 1259.6684761 s
			 val_= 10.2031619718, test_time  = 11.2603580952 s


epoch 4 : pseudo= -7.71768305834, cost= 18.4936895278, previous_cost= 19.3699461743, train_time = 1259.86650395 s
			 val_= 10.0689208305, test_time  = 11.2160029411 s


epoch 5 : pseudo= -7.84211264999, cost= 17.5660358633, previous_cost= 18.4936895278, train_time = 1260.40985894 s
			 val_= 10.1834330439, test_time  = 11.2208349705 s


epoch 6 : pseudo= -7.72274692776, cost= 17.1254339866, previous_cost= 17.5660358633, train_time = 1259.11413097 s
			 val_= 10.033324149, test_time

KeyboardInterrupt: 

# Load model

In [51]:
with open('model_phone_1.pkl', 'rb') as f:
    net5_see_see = cPickle.load(f)


In [48]:
import gc
gc.collect()

1153097

In [49]:
del model1